In [1]:
# run in engineering environment
import os
import pandas as pd
import numpy as np
import datetime
from google.cloud import bigquery
import geopandas
from shapely import wkt

Test with 10 rows

Import data for California only

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/amykim/Documents/Metis/Course_3_Engineering/Project/BigQuery/data-engineering-345807-a02ba30a47dd.json"
client = bigquery.Client()

QUERY = ('''

    SELECT * FROM `bigquery-public-data.utility_us.zipcode_area` z,`bigquery-public-data.noaa_lightning.lightning_*` l 
        WHERE ST_CONTAINS(ST_GeogFromText(z.zipcode_geom) , l.center_point_geom) and state_name = 'California'

    ''')

query_job = client.query(QUERY)  # API request
query_result = query_job.result() 
df= query_result.to_dataframe()

#1m 0.3s

Write to pickle

In [3]:
df.to_pickle('pickles/lightning_CA')

Write to csv

In [5]:
df=pd.read_pickle('pickles/lightning_CA')
df.shape
df.to_csv('lightning_CA.csv')

#1m 15.6s

### Import to SQLite using SQL Alchemy
Pipeline starts from here?

In [2]:
from sqlalchemy import create_engine

In [ ]:
df=pd.read_csv('lightning_CA.csv')

engine = create_engine('sqlite:///lightning.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "lightning_CA"
df.to_sql(sqlite_table,sqlite_connection,if_exists='replace')

#

In [5]:
from sqlalchemy import create_engine
import pandas as pd

def create_df():
    engine= create_engine('sqlite:///lightning.db')
    df=pd.read_sql(
    '''
    SELECT date, 
    zipcode, 
    city, 
    state_name as state, 
    number_of_strikes as count_lightning, 
    center_point_geom as center_point
    FROM lightning_CA
    '''
    ,engine)

    return df


In [6]:
df=create_df()
#1m 39.1s

Clean data

In [8]:
def clean_dates(df):    
    df['date']=pd.to_datetime(df['date'])
    df['year']=pd.DatetimeIndex(df['date']).year
    df['month']=pd.DatetimeIndex(df['date']).month
    return df


In [9]:
df=clean_dates(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283465 entries, 0 to 283464
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   date             283465 non-null  datetime64[ns]
 1   zipcode          283465 non-null  int64         
 2   city             283465 non-null  object        
 3   state            283465 non-null  object        
 4   count_lightning  283465 non-null  int64         
 5   center_point     283465 non-null  object        
 6   year             283465 non-null  int64         
 7   month            283465 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 17.3+ MB


In [11]:
df.city.unique()
df.groupby(["zipcode","year","month"]).sum("count_lightning").reset_index()

,zipcode,year,month,count_lightning
0,90027,1987,9,1
1,90027,1987,11,4
2,90027,1987,12,1
3,90027,1988,4,5
4,90027,1988,7,1
...,...,...,...,...
70134,96161,2018,7,33
70135,96161,2018,10,3
70136,96161,2020,6,17
70137,96161,2020,7,21


In [12]:
def clean_geo(df):
    df['center_point'] = geopandas.GeoSeries.from_wkt(df['center_point'])
    gdf = geopandas.GeoDataFrame(df,geometry='center_point')
    gdf['lon'] = gdf['center_point'].x
    gdf['lat'] = gdf['center_point'].y
    return gdf

In [13]:
gdf= clean_geo(df)
#24s

In [14]:
gdf

,date,zipcode,city,state,count_lightning,center_point,year,month,lon,lat
0,2020-03-08,96021,"Corning city, Rancho Tehama Reserve CDP, Richf...",California,8,POINT (-122.20000 39.90000),2020,3,-122.2,39.9
1,2020-08-11,96119,Madeline,California,8,POINT (-120.60000 41.00000),2020,8,-120.6,41.0
2,2020-08-16,94552,"Union City city, Hayward city, Sunol CDP, Fair...",California,8,POINT (-122.10000 37.80000),2020,8,-122.1,37.8
3,2020-08-16,94560,Newark city,California,8,POINT (-122.00000 37.50000),2020,8,-122.0,37.5
4,2020-08-17,96025,Dunsmuir city,California,8,POINT (-122.40000 41.20000),2020,8,-122.4,41.2
...,...,...,...,...,...,...,...,...,...,...
283460,1998-07-26,96132,Termo,California,4,POINT (-120.00000 41.00000),1998,7,-120.0,41.0
283461,2003-09-05,96132,Termo,California,8,POINT (-120.00000 40.90000),2003,9,-120.0,40.9
283462,2003-07-24,96132,Termo,California,14,POINT (-120.40000 40.90000),2003,7,-120.4,40.9
283463,1992-08-13,96132,Termo,California,5,POINT (-120.70000 41.00000),1992,8,-120.7,41.0


In [15]:
gdf.groupby(['lat','lon','year','month',])['count_lightning'].sum().reset_index()

,lat,lon,year,month,count_lightning
0,32.6,-117.0,1987,9,10
1,32.6,-117.0,1987,10,1
2,32.6,-117.0,1990,6,2
3,32.6,-117.0,1990,8,1
4,32.6,-117.0,1991,2,1
...,...,...,...,...,...
175312,42.0,-121.7,2017,9,1
175313,42.0,-121.7,2018,4,1
175314,42.0,-121.7,2018,5,5
175315,42.0,-121.7,2018,7,1


In [16]:
def center_point_summary(gdf):
    center_point_summary=gdf.groupby(['lat','lon','year','month',])['count_lightning'].sum().reset_index()
    return center_point_summary


In [20]:
center_point_summary=center_point_summary(gdf)


In [21]:
center_point_summary

,lat,lon,year,month,count_lightning
0,32.6,-117.0,1987,9,10
1,32.6,-117.0,1987,10,1
2,32.6,-117.0,1990,6,2
3,32.6,-117.0,1990,8,1
4,32.6,-117.0,1991,2,1
...,...,...,...,...,...
175312,42.0,-121.7,2017,9,1
175313,42.0,-121.7,2018,4,1
175314,42.0,-121.7,2018,5,5
175315,42.0,-121.7,2018,7,1


In [30]:
def summary_by_zipcode(df):
    summary_by_zipcode=df.groupby(["zipcode","year","month"]).sum("count_lightning").reset_index()
    return summary_by_zipcode

In [31]:
summary_by_zipcode=summary_by_zipcode(df)
summary_by_zipcode

,zipcode,year,month,count_lightning
0,90027,1987,9,1
1,90027,1987,11,4
2,90027,1987,12,1
3,90027,1988,4,5
4,90027,1988,7,1
...,...,...,...,...
70134,96161,2018,7,33
70135,96161,2018,10,3
70136,96161,2020,6,17
70137,96161,2020,7,21


In [25]:
def write_to_csv(summary):
    summary.to_csv('lightning].csv', index=False)
    return

In [26]:
write_to_csv(summary)

In [40]:
summary

,center_point,date,zipcode,count_lightning,year,month
0,POINT(-114.2 34.3),1987-09-05,92267,3,1987,9
1,POINT(-114.2 34.3),1987-09-23,92267,3,1987,9
2,POINT(-114.2 34.3),1987-09-24,92267,2,1987,9
3,POINT(-114.2 34.3),1987-11-01,92267,2,1987,11
4,POINT(-114.2 34.3),1988-06-17,92267,1,1988,6
...,...,...,...,...,...,...
283460,POINT(-124.3 40.6),2017-01-09,95536,3,2017,1
283461,POINT(-124.3 40.6),2017-01-19,95536,1,2017,1
283462,POINT(-124.3 40.6),2017-03-06,95536,1,2017,3
283463,POINT(-124.3 40.6),2017-08-10,95536,6,2017,8
